
# 📦 긴급도판별 — 훈련용 **MTL 전처리** (Colab)
**목표:** 하나의 데이터셋으로 **긴급도(urgency)**와 **감정(sentiment)**을 **동시에 학습**할 수 있도록 전처리합니다.

## 입력
- `Processed_Data/calls_df_preprocessed.feather`
- `Processed_Data/utts_df_preprocessed.feather`  *(이전 EDA 노트북에서 생성됨)*

## 출력
- `MTL_ready/` 폴더 아래에 **CSV + Parquet** 동시 저장
  - `X_train / X_test` (입력 특징: `full_text`, `turn_count`, `initial_response_ms`)
  - `y_train_urgency / y_test_urgency` (하=0, 중=1, 상=2)
  - `y_train_sentiment / y_test_sentiment` (기타부정=0, 당황/난처=1, 불안/걱정=2, 중립=3)

## 원칙
- **타깃 누설 방지:** 감정은 예측 대상이므로 입력 피처에 포함하지 않음
- **그룹 누수 최소화:** 동일 `recordId`는 같은 분할에 들어가도록 인덱스 보관
- **재현성:** CSV만으로도 학습 가능, Parquet은 고속/안정/재현 용도로 추가 저장


In [ ]:

# ⬇️ 1) Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# 📁 2) 경로 설정
import os
PROCESSED_DIR = "/content/drive/MyDrive/project1/Processed_Data"
OUT_DIR = os.path.join(PROCESSED_DIR, "MTL_ready")
os.makedirs(OUT_DIR, exist_ok=True)
print("INPUT :", PROCESSED_DIR)
print("OUTPUT:", OUT_DIR)


INPUT : /content/drive/MyDrive/project1/Processed_Data
OUTPUT: /content/drive/MyDrive/project1/Processed_Data/MTL_ready


In [ ]:

# 📥 3) Feather 로드
import pandas as pd

calls_fp = os.path.join(PROCESSED_DIR, "calls_df_preprocessed.feather")
utts_fp  = os.path.join(PROCESSED_DIR, "utts_df_preprocessed.feather")

calls_df = pd.read_feather(calls_fp)
utts_df  = pd.read_feather(utts_fp)

print(f"✅ 로드 완료 — calls_df: {calls_df.shape}, utts_df: {utts_df.shape}")
display(calls_df.head(2))
display(utts_df.head(2))


✅ 로드 완료 — calls_df: (127178, 13), utts_df: (3689183, 6)


,recordId,label_path,gender,sentiment,disasterLarge,disasterMedium,urgencyLevel,n_utterances,startAt,endAt,turn_count,initial_response_ms,caller_talk_ratio
0,9adf51433e0a811a96eb,/content/drive/MyDrive/project1/Training/2.라벨링...,M,중립,기타,기타,하,23,0,43960.0,18,NaN,0.486810
1,f963d97b5a46fa324393,/content/drive/MyDrive/project1/Training/2.라벨링...,M,당황/난처,기타,기타,하,17,0,33280.0,14,NaN,0.499082


,recordId,speaker,startAt,endAt,text,duration_ms
0,9adf51433e0a811a96eb,0,1339,2576,119입니다.,1237
1,9adf51433e0a811a96eb,0,3555,4382,여보세요.,827


In [ ]:

# 🧩 4) 통화 전체 텍스트 병합(full_text)
full_texts = (
    utts_df
      .sort_values(["recordId","startAt"])
      .groupby("recordId")["text"]
      .apply(lambda s: " ".join(map(str, s.dropna())))
      .reset_index()
      .rename(columns={"text":"full_text"})
)

# 기본 피처 + 타깃 후보만 추출
feature_cols = ["recordId", "turn_count", "initial_response_ms", "urgencyLevel", "sentiment"]
base = calls_df[feature_cols].copy()

df = (base.astype({"recordId": str})
          .merge(full_texts.astype({"recordId": str}), on="recordId", how="inner"))
print("✅ 텍스트 병합:", df.shape)
display(df.head(3))


✅ 텍스트 병합: (127178, 6)


,recordId,turn_count,initial_response_ms,urgencyLevel,sentiment,full_text
0,9adf51433e0a811a96eb,18,NaN,하,중립,119입니다. 여보세요. 네 수고하십니다. 네 네 저기저 속보기 테스트 하려고 전화...
1,f963d97b5a46fa324393,14,NaN,하,당황/난처,"예, 여보세요. 네, 119입니다. 예, 여기 남구 방림동 휴먼시아인데요. 네, 방..."
2,5be0d53170f311622ad9,38,NaN,하,당황/난처,"그 저 번지수는 모르세요 혹시? 119입니다. 아 네, 여기 저기 여기 양림동인데요..."


In [ ]:

# 🧼 5) 결측/타입 클린업 + 타깃 인코딩
import numpy as np

# 기본 결측 처리
df["turn_count"] = df["turn_count"].fillna(df["turn_count"].median())
med_resp = df["initial_response_ms"].median()
df["initial_response_ms"] = df["initial_response_ms"].fillna(med_resp)

# 타깃 인코딩
urgency_map = {"하":0, "중":1, "상":2}
sentiment_map = {"기타부정":0, "당황/난처":1, "불안/걱정":2, "중립":3}

df["y_urgency"]   = df["urgencyLevel"].map(urgency_map)
df["y_sentiment"] = df["sentiment"].map(sentiment_map)

# 라벨 결측 제거
df = df.dropna(subset=["y_urgency", "y_sentiment"]).copy()
df["y_urgency"]   = df["y_urgency"].astype("int64")
df["y_sentiment"] = df["y_sentiment"].astype("int64")

# 🔒 입력 X에서 타깃 관련 컬럼 제거 (누설 방지)
X = df[["turn_count", "initial_response_ms", "full_text"]].copy()
y_u = df["y_urgency"].copy()
y_s = df["y_sentiment"].copy()

print("✅ 피처/타깃 확정")
print("X columns:", list(X.columns))
print("y_urgency classes:", sorted(y_u.unique()))
print("y_sentiment classes:", sorted(y_s.unique()))


✅ 피처/타깃 확정
X columns: ['turn_count', 'initial_response_ms', 'full_text']
y_urgency classes: [np.int64(0), np.int64(1), np.int64(2)]
y_sentiment classes: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)]


In [ ]:

# 🔀 6) Train/Test 분할 (두 타깃 조합으로 stratify)
from sklearn.model_selection import train_test_split

combo = df["y_urgency"].astype(str) + "_" + df["y_sentiment"].astype(str)

X_train, X_test, y_train_u, y_test_u, y_train_s, y_test_s = train_test_split(
    X, y_u, y_s,
    test_size=0.2,
    random_state=42,
    stratify=combo
)

# (선택) recordId 인덱스 보관 — 동일 통화 누수 점검용
idx_train = X_train.index.to_series().map(df["recordId"])
idx_test  = X_test.index.to_series().map(df["recordId"])

print("✅ 분할 완료")
print("Train:", X_train.shape, "| Test:", X_test.shape)
print("Train recordId 샘플:", idx_train.head().tolist())


✅ 분할 완료
Train: (101742, 3) | Test: (25436, 3)
Train recordId 샘플: ['a8c652941aef654f771b', '15867cb49eaeb62bec88', 'fbfaefd40cfabb88375a', '2976a428e935d65003ea', '7f16f979b0ae680043fc']


In [ ]:

# 💾 7) 저장 (CSV + Parquet)
os.makedirs(OUT_DIR, exist_ok=True)

# CSV
X_train.to_csv(os.path.join(OUT_DIR, "X_train.csv"), index=False)
X_test.to_csv(os.path.join(OUT_DIR, "X_test.csv"), index=False)
y_train_u.to_csv(os.path.join(OUT_DIR, "y_train_urgency.csv"), index=False)
y_test_u.to_csv(os.path.join(OUT_DIR, "y_test_urgency.csv"), index=False)
y_train_s.to_csv(os.path.join(OUT_DIR, "y_train_sentiment.csv"), index=False)
y_test_s.to_csv(os.path.join(OUT_DIR, "y_test_sentiment.csv"), index=False)

# Parquet
X_train.to_parquet(os.path.join(OUT_DIR, "X_train.parquet"), index=False)
X_test.to_parquet(os.path.join(OUT_DIR, "X_test.parquet"), index=False)
y_train_u.to_frame("y_urgency").to_parquet(os.path.join(OUT_DIR, "y_train_urgency.parquet"), index=False)
y_test_u.to_frame("y_urgency").to_parquet(os.path.join(OUT_DIR, "y_test_urgency.parquet"), index=False)
y_train_s.to_frame("y_sentiment").to_parquet(os.path.join(OUT_DIR, "y_train_sentiment.parquet"), index=False)
y_test_s.to_frame("y_sentiment").to_parquet(os.path.join(OUT_DIR, "y_test_sentiment.parquet"), index=False)

print("✅ 저장 완료 →", OUT_DIR)
print("📄 파일 목록:")
for f in sorted(os.listdir(OUT_DIR)):
    if f.endswith((".csv",".parquet")):
        print(" -", f)


✅ 저장 완료 → /content/drive/MyDrive/project1/Processed_Data/MTL_ready
📄 파일 목록:
 - X_test.csv
 - X_test.parquet
 - X_train.csv
 - X_train.parquet
 - y_test_sentiment.csv
 - y_test_sentiment.parquet
 - y_test_urgency.csv
 - y_test_urgency.parquet
 - y_train_sentiment.csv
 - y_train_sentiment.parquet
 - y_train_urgency.csv
 - y_train_urgency.parquet


In [ ]:

# 🔎 8) 간단 무결성 점검
import numpy as np

def class_dist(y, name):
    vc = y.value_counts(normalize=True).sort_index()
    print(f"{name} class dist:", {int(k): round(float(v), 3) for k, v in vc.items()})

class_dist(y_train_u, "y_train_urgency")
class_dist(y_test_u,  "y_test_urgency")
class_dist(y_train_s, "y_train_sentiment")
class_dist(y_test_s,  "y_test_sentiment")

assert set(y_train_u.unique()).issubset({0,1,2})
assert set(y_test_u.unique()).issubset({0,1,2})
assert set(y_train_s.unique()).issubset({0,1,2,3})
assert set(y_test_s.unique()).issubset({0,1,2,3})

print("✅ 무결성 OK")


y_train_urgency class dist: {0: 0.354, 1: 0.337, 2: 0.309}
y_test_urgency class dist: {0: 0.354, 1: 0.337, 2: 0.309}
y_train_sentiment class dist: {0: 0.023, 1: 0.376, 2: 0.536, 3: 0.066}
y_test_sentiment class dist: {0: 0.023, 1: 0.376, 2: 0.536, 3: 0.066}
✅ 무결성 OK
